# Исследование причин неэффективности привлечения пользователей приложения.

Развлекательное приложение Procrastinate Pro+ несет убытки в течение нескольких прошлых месяцев. Деньги, вложенные в привлечение пользователей не увеличивают прибыльность.

В нашем распоряжении есть лог сервера с данными о посещениях приложения новыми пользователями, зарегистрировавшимися в период с 2019-05-01 по 2019-10-27, выгрузка их покупок за этот период, а также статистика рекламных расходов. Нужно изучить, как люди пользуются продуктом, когда они начинают покупать, сколько денег приносит каждый клиент, когда он окупается и какие факторы отрицательно влияют на привлечение пользователей.

Необходимо найти возможные причины обнаруженных проблем и сформировать рекомендации для отдела маркетинга для повышения эффективности.

## Загрузка данных и подготовка к анализу

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import datetime, timedelta
from scipy import stats as st

%config InlineBackend.figure_format = 'retina'

In [2]:
visits = pd.read_csv('/datasets/visits_info_short.csv')
visits

FileNotFoundError: ignored

In [ ]:
visits.info()

In [ ]:
visits.columns = [x.lower().replace(' ', '_') for x in visits.columns]
visits['session_start'] = pd.to_datetime(visits['session_start'])
visits['session_end'] = pd.to_datetime(visits['session_end'])

In [ ]:
visits[visits.duplicated()]

In [ ]:
visits[visits.duplicated(subset=['user_id', 'session_end'])]

In [ ]:
orders = pd.read_csv('/datasets/orders_info_short.csv')
orders

In [ ]:
orders.info()

In [ ]:
orders.columns = [x.lower().replace(' ', '_') for x in orders.columns]
orders['event_dt'] = pd.to_datetime(orders['event_dt'])

In [ ]:
orders[orders.duplicated()]

In [ ]:
orders[orders.duplicated(subset=['user_id', 'event_dt'])]

In [ ]:
costs = pd.read_csv('/datasets/costs_info_short.csv')
costs

In [ ]:
costs.info()

In [ ]:
costs.columns = [x.lower() for x in costs.columns]
costs['dt'] = pd.to_datetime(costs['dt']).dt.date

In [ ]:
costs.info()

In [ ]:
costs[costs.duplicated()]

Загрузили таблицы для работы, изменили тип данных в датах для удобства работы, привели к нижнему регистру названия колонок. Проверили наличие дубликатов.

## Функции для расчета и анализа LTV, ROI, удержания и конверсии

### Функция для создания пользовательских профилей

In [ ]:
# функция для создания пользовательских профилей

def get_profiles(visits, orders, costs):

    profiles = (
        visits.sort_values(by=['user_id', 'session_start'])
        .groupby('user_id')
        .agg(
            {
                'session_start': 'first',
                'channel': 'first',
                'device': 'first',
                'region': 'first',
            }
        )
        .rename(columns={'session_start': 'first_ts'})
        .reset_index()
    )
    profiles['dt'] = profiles['first_ts'].dt.date
    profiles['month'] = profiles['first_ts'].astype('datetime64[M]')
    profiles['payer'] = profiles['user_id'].isin(orders['user_id'].unique())
    new_users = (
        profiles.groupby(['dt', 'channel'])
        .agg({'user_id': 'nunique'})
        .rename(columns={'user_id': 'unique_users'})
        .reset_index()
    )
    costs = costs.merge(new_users, on=['dt', 'channel'], how='left')
    costs['acquisition_cost'] = costs['costs'] / costs['unique_users']
    profiles = profiles.merge(
        costs[['dt', 'channel', 'acquisition_cost']],
        on=['dt', 'channel'],
        how='left',
    )
    profiles['acquisition_cost'] = profiles['acquisition_cost'].fillna(0)
    return profiles

### Функция для расчёта удержания

In [ ]:
# функция для расчёта удержания

def get_retention(
    profiles,
    sessions,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):
    dimensions = ['payer'] + dimensions
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    result_raw = result_raw.merge(
        sessions[['user_id', 'session_start']], on='user_id', how='left'
    )
    result_raw['lifetime'] = (
        result_raw['session_start'] - result_raw['first_ts']
    ).dt.days

    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result

    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    return result_raw, result_grouped, result_in_time 

### Функция для расчёта конверсии

In [ ]:
# функция для расчёта конверсии

def get_conversion(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    first_purchases = (
        purchases.sort_values(by=['user_id', 'event_dt'])
        .groupby('user_id')
        .agg({'event_dt': 'first'})
        .reset_index()
    )

    result_raw = result_raw.merge(
        first_purchases[['user_id', 'event_dt']], on='user_id', how='left'
    )

    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days

    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users' 
        dimensions = dimensions + ['cohort']

    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        result = result.fillna(0).cumsum(axis = 1)
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)       
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)
    if 'cohort' in dimensions: 
        dimensions = []
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )
    return result_raw, result_grouped, result_in_time 

### Функция для расчёта LTV и ROI

In [ ]:
# функция для расчёта LTV и ROI

def get_ltv(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')  
    result_raw = result_raw.merge(
        purchases[['user_id', 'event_dt', 'revenue']], on='user_id', how='left'
    )
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days

    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users'
        dimensions = dimensions + ['cohort']
    def group_by_dimensions(df, dims, horizon_days):        
        result = df.pivot_table(
            index=dims, columns='lifetime', values='revenue', aggfunc='sum'
        )     
        result = result.fillna(0).cumsum(axis=1)      
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )      
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)       
        result = result.div(result['cohort_size'], axis=0)        
        result = result[['cohort_size'] + list(range(horizon_days))]       
        result['cohort_size'] = cohort_sizes
        cac = df[['user_id', 'acquisition_cost'] + dims].drop_duplicates()
        cac = (
            cac.groupby(dims)
            .agg({'acquisition_cost': 'mean'})
            .rename(columns={'acquisition_cost': 'cac'})
        )
        roi = result.div(cac['cac'], axis=0)
        roi = roi[~roi['cohort_size'].isin([np.inf])]
        roi['cohort_size'] = cohort_sizes
        roi['cac'] = cac['cac']
        roi = roi[['cohort_size', 'cac'] + list(range(horizon_days))]
        return result, roi

    result_grouped, roi_grouped = group_by_dimensions(
        result_raw, dimensions, horizon_days
    )
    if 'cohort' in dimensions:
        dimensions = []
    result_in_time, roi_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    return (
        result_raw,  
        result_grouped,  
        result_in_time, 
        roi_grouped, 
        roi_in_time, 
    )

### Функция для сглаживания фрейма

In [ ]:
# функция для сглаживания фрейма

def filter_data(df, window):
    # для каждого столбца применяем скользящее среднее
    for column in df.columns.values:
        df[column] = df[column].rolling(window).mean() 
    return df 

### Функция для визуализации удержания

In [ ]:
# функция для визуализации удержания

def plot_retention(retention, retention_history, horizon, window=7):

    plt.figure(figsize=(15, 10))

    retention = retention.drop(columns=['cohort_size', 0])  
    retention_history = retention_history.drop(columns=['cohort_size'])[
        [horizon - 1]
    ]
    if retention.index.nlevels == 1:
        retention['cohort'] = 'All users'
        retention = retention.reset_index().set_index(['cohort', 'payer'])

    ax1 = plt.subplot(2, 2, 1)
    retention.query('payer == True').droplevel('payer').T.plot(
        grid=True, ax=ax1
    )
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Удержание платящих пользователей')

    ax2 = plt.subplot(2, 2, 2, sharey=ax1)
    retention.query('payer == False').droplevel('payer').T.plot(
        grid=True, ax=ax2
    )
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Удержание неплатящих пользователей')

    ax3 = plt.subplot(2, 2, 3)
    
    columns = [
        name
        for name in retention_history.index.names
        if name not in ['dt', 'payer']
    ]
   
    filtered_data = retention_history.query('payer == True').pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax3)
    plt.xlabel('Дата привлечения')
    plt.title(
        'Динамика удержания платящих пользователей на {}-й день'.format(
            horizon
        )
    )   
    ax4 = plt.subplot(2, 2, 4, sharey=ax3)
    filtered_data = retention_history.query('payer == False').pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax4)
    plt.xlabel('Дата привлечения')
    plt.title(
        'Динамика удержания неплатящих пользователей на {}-й день'.format(
            horizon
        )
    )
    
    plt.tight_layout()
    plt.show()

### Функция для визуализации конверсии

In [ ]:
# функция для визуализации конверсии

def plot_conversion(conversion, conversion_history, horizon, window=7):
    plt.figure(figsize=(15, 5))
    conversion = conversion.drop(columns=['cohort_size'])
  
    conversion_history = conversion_history.drop(columns=['cohort_size'])[
        [horizon - 1]
    ]
    ax1 = plt.subplot(1, 2, 1)
    conversion.T.plot(grid=True, ax=ax1)
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Конверсия пользователей')

    ax2 = plt.subplot(1, 2, 2, sharey=ax1)
    columns = [
       
        name for name in conversion_history.index.names if name not in ['dt']
    ]
    filtered_data = conversion_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax2)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика конверсии пользователей на {}-й день'.format(horizon))

    plt.tight_layout()
    plt.show() 

### Функция для визуализации LTV и ROI

In [ ]:
# функция для визуализации LTV и ROI

def plot_ltv_roi(ltv, ltv_history, roi, roi_history, horizon, window=7):

    plt.figure(figsize=(20, 10))

    ltv = ltv.drop(columns=['cohort_size'])
    
    ltv_history = ltv_history.drop(columns=['cohort_size'])[[horizon - 1]]
    cac_history = roi_history[['cac']]
    roi = roi.drop(columns=['cohort_size', 'cac'])   
    roi_history = roi_history.drop(columns=['cohort_size', 'cac'])[
        [horizon - 1]
    ]

    ax1 = plt.subplot(2, 3, 1)
    ltv.T.plot(grid=True, ax=ax1)
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('LTV')

    ax2 = plt.subplot(2, 3, 2, sharey=ax1)
  
    columns = [name for name in ltv_history.index.names if name not in ['dt']]
    filtered_data = ltv_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax2)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика LTV пользователей на {}-й день'.format(horizon))

  
    ax3 = plt.subplot(2, 3, 3, sharey=ax1)
    
    columns = [name for name in cac_history.index.names if name not in ['dt']]
    filtered_data = cac_history.pivot_table(
        index='dt', columns=columns, values='cac', aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax3)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика стоимости привлечения пользователей')
   
    ax4 = plt.subplot(2, 3, 4)
    roi.T.plot(grid=True, ax=ax4)
    plt.axhline(y=1, color='red', linestyle='--', label='Уровень окупаемости')
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('ROI')

   
    ax5 = plt.subplot(2, 3, 5, sharey=ax4)
   
    columns = [name for name in roi_history.index.names if name not in ['dt']]
    filtered_data = roi_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax5)
    plt.axhline(y=1, color='red', linestyle='--', label='Уровень окупаемости')
    plt.xlabel('Дата привлечения')
    plt.title('Динамика ROI пользователей на {}-й день'.format(horizon))

    plt.tight_layout()
    plt.show() 

## Исследовательский анализ данных

Построим профили пользователей, определим минимальную и максимальную дату привлечения пользователей.
Выясним:
- Из каких стран приходят посетители? Какие страны дают больше всего платящих пользователей?
- Какими устройствами они пользуются? С каких устройств чаще всего заходят платящие пользователи?
- По каким рекламным каналам шло привлечение пользователей? Какие каналы приносят больше всего платящих пользователей?

In [ ]:
profiles = get_profiles(visits, orders, costs)
profiles.head()

In [ ]:
profiles.info()

In [ ]:
profiles['first_ts'].min()

In [ ]:
profiles['first_ts'].max()

In [ ]:
observation_date = datetime(2019,11,1)
horizon_days = 14
last_suitable_acquisition_date = observation_date - timedelta(days = horizon_days-1)
pd.to_datetime(last_suitable_acquisition_date)

Самая ранняя дата регистрации 01 мая 2019. В нашем датафрейме самая поздняя дата регистрации - это 27 октября 2019 года, но даты позже 19 октября мы не будем брать в рассмотрение, т.к. наш анализ горизонта 14 дней, а сегодня 01 ноября 2019 года. 

### Анализ данных по региональному признаку

In [ ]:
profiles.query('payer == True')['user_id'].count()

In [ ]:
profiles['region'].value_counts()

In [ ]:
profiles.query('payer == True')['region'].value_counts()

In [ ]:
profiles.query('payer == False')['region'].value_counts()

In [ ]:
reg_tbl = profiles.groupby('region').agg({'user_id':'nunique','payer':['sum','mean']})
reg_tbl.columns = ['users', 'payers', 'payers,%']
reg_tbl['payers,%'] = round(reg_tbl['payers,%']*100,2)
reg_tbl.sort_values(by='users',ascending=False)

In [ ]:
profiles['region'].value_counts()/profiles['user_id'].count()*100

In [ ]:
profiles.query('payer == True')['region'].value_counts()/profiles.query('payer == True')['user_id'].count()*100

Америка - абсолютный лидер по количеству пользователей (66,7%) и по платящим пользователям (77,7%). Оттуда у нас 100 тыс.пользователей, из которых 6,9% совершают покупки. Великобритания, Франция и Германия дают гораздо меньше пользователей, покупателями становятся примерно 4% пользователей, в Германии этот показатель выше, чем у остальных.

### Анализ устройств, используемых пользователей

In [ ]:
profiles['device'].value_counts()

In [ ]:
profiles.query('payer == True')['device'].value_counts()

In [ ]:
profiles.query('payer == False')['device'].value_counts()

In [ ]:
dev_tbl = profiles.groupby('device').agg({'user_id':'nunique','payer':['sum','mean']})
dev_tbl.columns = ['users', 'payers', 'payers,%']
dev_tbl['payers,%'] = round(dev_tbl['payers,%']*100,2)
dev_tbl.sort_values(by='users',ascending=False)

In [ ]:
profiles['device'].value_counts()/profiles['device'].count()*100

In [ ]:
profiles.query('payer == True')['device'].value_counts()/profiles.query('payer == True')['user_id'].count()*100

### Анализ источников траффика

Больше всего пользователей заходят в приложение с iPhone (36,3%), они же наибольшая численная и относительная часть плательщиков (38,1%). В процентном соотношении мы видим, что пользователи Mac чаще остальных становятся покупателями (6,36%) и приносят 21,5% покупок. 

In [ ]:
profiles['channel'].value_counts()

In [ ]:
profiles.query('payer == True')['channel'].value_counts()

In [ ]:
profiles.query('payer == False')['channel'].value_counts()

In [ ]:
chan_tbl = profiles.groupby('channel').agg({'user_id':'nunique','payer':['sum','mean']})
chan_tbl.columns = ['users', 'payers', 'payers,%']
chan_tbl['payers,%'] = round(chan_tbl['payers,%']*100,2)
chan_tbl.sort_values(by='users',ascending=False)

In [ ]:
profiles['channel'].value_counts()/profiles['channel'].count()*100

In [ ]:
profiles.query('payer == True')['channel'].value_counts()/profiles.query('payer == True')['user_id'].count()*100

Больше всего пользователей (37%) приводит к нам естественный траффик, они очень редко (2,06% случаев) становятся плательщиками, но занимают третье место среди всех платящих пользователей (13%). Мы анализируем именно привлечение пользователей, на которое были потрачены деньги. На привлечение пользователей из источника "organic" не тратили ничего, пользователи приходили сами.

Среди источников, на коткорые были выделены рекламные средства, у платящих на первом месте Faceboom (40%), затем TipTop (21%), остальные приносят меньше 5% покупателей.

## Маркетинг

Выясним:
- Сколько денег потратили? Всего / на каждый источник / по времени
- Сколько в среднем стоило привлечение одного покупателя из каждого источника?

In [ ]:
observation_date = datetime(2019,11,1).date()
horizon_days = 14
costs['month'] = costs['dt'].astype('datetime64[M]')

In [ ]:
costs

In [ ]:
costs['costs'].sum()

In [ ]:
costs.groupby('channel')['costs'].sum().sort_values(ascending=False)

In [ ]:
costs.groupby('month')['costs'].sum()

In [ ]:
costs.groupby('dt')['costs'].sum().plot(figsize=(12,5))

In [ ]:
plt.figure()
costs.groupby('dt')['costs'].sum().plot(title='Затраты на рекламу за все время', ax=plt.subplot(), figsize=(12,5))
plt.xlabel('Год, месяц')
plt.ylabel('Затраты, руб.')
plt.show()

<div class="alert alert-danger">
<b>Комментарий ревьюера:</b>☁️

Давай доработаем графики, добавь, пожалуйста, заголовок и подписи осей. ЧТобы избавиться от `<AxesSubplot:xlabel='dt'>
`, можно поставить `;` в конце строки или же написать `plt.show()`
</div>


<div class="alert alert-info"> Добавила исправленный график под неправильным </div>

<div class="alert alert-success">
<b>Комментарий ревьюера v2:</b>☀️
    
Отлично и ниже по работе тоже графики изменены
</div>


In [ ]:
profiles.groupby('month')['user_id'].count()

In [ ]:
profiles.groupby('month')['user_id'].count().mean()

Всего на рекламу было потрачено 105497 руб. Больше половины этой суммы (54751 руб.) ушло в TipTop, в FaceBoom заплатили 32445 руб., на остальные источники потрачены было 5 и менее тыс. руб. 

Количество новых пользователей в среднем 25 тыс. чел., наибольший приток был в мае, когда затраты на рекламу были самыми низкими. Рекламный бюджет от месяца к месяцу рос до сентября, затем в октябре немного снижен.

Добавим в таблицу данные о выручке с пользователей и продолжим анализ.

In [ ]:
user_price = profiles.groupby('channel').agg({'user_id':'nunique','payer':'sum','acquisition_cost':'sum'})
user_price['cost_per_user'] = user_price['acquisition_cost']/user_price['user_id']
user_price['cost_per_payer'] = user_price['acquisition_cost']/user_price['payer']
user_price.sort_values(by='acquisition_cost', ascending=False)

In [ ]:
orders_sum = orders.groupby('user_id')['revenue'].sum()
profiles_with_orders = profiles.merge(orders_sum, on='user_id', how='left')
profiles_with_orders = profiles_with_orders.fillna(0)
profiles_with_orders

Проверим, правильно ли сосчитана выручка.

In [ ]:
profiles_with_orders['revenue'].sum()

In [ ]:
orders['revenue'].sum()

In [ ]:
user_revenue = profiles_with_orders.groupby('channel').agg({'user_id':'nunique','payer':'sum','acquisition_cost':'sum','revenue':'sum'})
user_revenue['cost_per_user'] = user_revenue['acquisition_cost']/user_revenue['user_id']
user_revenue['cost_per_payer'] = user_revenue['acquisition_cost']/user_revenue['payer']
user_revenue['revenue_per_payer'] = user_revenue['revenue']/user_revenue['payer']
user_revenue.sort_values(by='revenue_per_payer', ascending=False)

Если посмотреть на показатель выручки на одного платящего пользователя, то  TipTop и FaceBoom, на которые ушла львиная часть рекламного бюджета точно не самые результативные.  А FaceBoom вообще на последней строчке, стоимость привлечения одного платящего пользователя здесь составила 9.12, а выручка с платящего пользователя 7.2, мы видим явный убыток. 

"Качество" платящих пользователей рассмотрим на гистограммах для трех самых прибыльных источниках траффика и для сравнения FaceBoom.

In [ ]:
profiles_with_orders.query('channel == "WahooNetBanner" & payer == True')['revenue'].hist(label='55')

In [ ]:
plt.figure()
profiles_with_orders.query('channel == "WahooNetBanner" & payer == True')['revenue'].plot(title='Выручка с пользователя с канала WahooNetBanner', kind='hist', ax=plt.subplot());
plt.ylabel('Выручка, руб.')
plt.show()

In [ ]:
profiles_with_orders.query('channel == "lambdaMediaAds" & payer == True')['revenue'].hist()

In [ ]:
plt.figure()
profiles_with_orders.query('channel == "lambdaMediaAds" & payer == True')['revenue'].plot(title='Выручка с пользователя с канала lambdaMediaAds', kind='hist', ax=plt.subplot());
plt.ylabel('Выручка, руб.')
plt.show()

In [ ]:
profiles_with_orders.query('channel == "YRabbit" & payer == True')['revenue'].hist()

In [ ]:
plt.figure()
profiles_with_orders.query('channel == "YRabbit" & payer == True')['revenue'].plot(title='Выручка с пользователя с канала YRabbit', kind='hist', ax=plt.subplot());
plt.ylabel('Выручка, руб.')
plt.show()

In [ ]:
profiles_with_orders.query('channel == "FaceBoom" & payer == True')['revenue'].hist()

In [ ]:
plt.figure()
profiles_with_orders.query('channel == "FaceBoom" & payer == True')['revenue'].plot(title='Выручка с пользователя с канала FaceBoom', kind='hist', ax=plt.subplot());
plt.ylabel('Выручка, руб.')
plt.show()

Пользователь с WahooNetBanner чаще всего тратит в приложении 50 рублей, с YRabbit от 10 до 50, с lambdaMediaAds от 20 до 50 рублей. Плательщик с FaceBoom тратит от 5 до 12 рублей.

Посмотрим распределенеие расходов на рекламу и доходов от продаж по регионам.

In [ ]:
profiles_with_orders.groupby('region').agg({'acquisition_cost':'sum', 'revenue':'sum'}).sort_values(by='acquisition_cost', ascending=False)

In [ ]:
line_1 = profiles_with_orders.query('region == "United States"').groupby('dt')['acquisition_cost'].sum()
line_2 = profiles_with_orders.query('region == "France"').groupby('dt')['acquisition_cost'].sum()
line_3 = profiles_with_orders.query('region == "UK"').groupby('dt')['acquisition_cost'].sum()
line_4 = profiles_with_orders.query('region == "Germany"').groupby('dt')['acquisition_cost'].sum()

fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot()
 
ax1.plot(line_1, color = 'green', label = 'USA') 
ax1.plot(line_2, color = 'red', label = 'FR') 
ax1.plot(line_3, color = 'blue', label = 'UK') 
ax1.plot(line_4, color = 'black', label = 'GR') 
ax1.legend() 
ax1.set_title('Расходы на рекламу по регионам')
plt.show() 

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>☀️
    
Для того, чтобы внимательнее изучить колебания расходов по регионам можно строить несколько графиков по размеру значений, здесь это не так критично, но вообще не особо понятно, как там дела с бюджетом всех стран кроме US
</div>

In [ ]:
line_1 = profiles_with_orders.query('channel != "organic" & region == "United States"').groupby('dt')['revenue'].sum()
line_2 = profiles_with_orders.query('channel != "organic" & region == "France"').groupby('dt')['revenue'].sum()
line_3 = profiles_with_orders.query('channel != "organic" & region == "UK"').groupby('dt')['revenue'].sum()
line_4 = profiles_with_orders.query('channel != "organic" & region == "Germany"').groupby('dt')['revenue'].sum()

fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot()
 
ax1.plot(line_1, color = 'green', label = 'USA') 
ax1.plot(line_2, color = 'red', label = 'FR') 
ax1.plot(line_3, color = 'blue', label = 'UK') 
ax1.plot(line_4, color = 'black', label = 'GR') 
ax1.legend() 
ax1.set_title('Доходы по регионам')
plt.show() 

In [ ]:
rev_reg = profiles_with_orders.query('channel != "organic"').groupby('region').agg({'acquisition_cost':'sum', 'revenue':'sum'}).sort_values(by='acquisition_cost', ascending=False)
rev_reg['acquisition_cost'] = round(rev_reg['acquisition_cost'],2)
rev_reg['revenue/cost'] = rev_reg['revenue'] / rev_reg['acquisition_cost']
rev_reg

Анализ затрат на рекламу по регионам выявил, что 86% расходов на рекламу уходят на привлечение пользователей в США. Графики показывают, что расходы на рекламу в Америке растут постоянно, а вот доходы в этом регионе в целом снижаются. 

Исключив из анализа доходы от пользователей "organic", на которых расходов на рекламу не было, мы получаем следующий результат: во Франции, Великобритании и Германии 1 рубль, потраченный на рекламу, приносит 3.27 - 3.75 рублей дохода, а в США только 1.31 руб. 

Явно реклама в Америке не приносит ожидаемой прибыли. Рассмотрим этот регион в разрезе платформ, на которых размещалась реклама. 

In [ ]:
usa_channel = (profiles_with_orders.query('region == "United States" & channel != "organic"').groupby('channel').agg({'acquisition_cost':'sum','revenue':'sum'}).
sort_values(by='acquisition_cost', ascending=False))
usa_channel['revenue/cost'] = usa_channel['revenue'] / usa_channel['acquisition_cost']
usa_channel

Из пяти платформ, на которых размещалась реклама в Америке, на первый взгляд, хорошие результаты показывают те три, затраты на которые были довольно низкими. А вот TipTop и FaceBoom, самые затратные, показывают очень низкий результат по платящим пользователям и их расходам в приложении.

Ниже рассмотрим показатели LTV, CAC и ROI по всем данным.

In [ ]:
(profiles.query('channel != "organic"').pivot_table(index='month', columns='channel', values='acquisition_cost', aggfunc='mean').
 style.background_gradient(axis=0, cmap='Purples'))

In [ ]:
(profiles_with_orders.query('channel != "organic"').pivot_table(index='month', columns='channel', values='revenue', aggfunc='mean').
 style.background_gradient(axis=0, cmap='Purples'))

## Окупаемость рекламы для привлечения пользователей

### Общая окупаемость рекламы

Проанализируем окупаемость рекламы в общем с помощью LTV и ROI.

In [ ]:
profiles = profiles.query('channel != "organic"')
observation_date = datetime(2019,11,1).date()
horizon_days = 14

# считаем LTV и ROI
ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days
)

# строим графики
plot_ltv_roi(ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days) 

In [ ]:
ltv_grouped.drop(columns=['cohort_size',0]).reset_index()


In [ ]:
roi_grouped.drop(columns=['cohort_size',0]).reset_index()

По графикам можно сделать такие выводы:
Реклама не окупается. ROI в конце периода на уровне 80%. CAC растет, расходы на рекламу увеличивались почти весь анализируемый период. График динамики ROI показывает, что на уровне окупаемости расходы на рекламу не были с середины июля. Чтобы разобраться в причинах, пройдём по всем доступным характеристикам пользователей — стране, источнику и устройству первого посещения.

### Окупаемость рекламы с разбивкой по устройствам

In [ ]:
profiles = profiles.query('channel != "organic"')
dimensions = ['device']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14
) 

In [ ]:
# смотрим конверсию с разбивкой по устройствам

conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_conversion(conversion_grouped, conversion_history, horizon_days) 

In [ ]:
# смотрим удержание с разбивкой по устройствам

retention_raw, retention_grouped, retention_history = get_retention(
    profiles, visits, observation_date, horizon_days, dimensions=dimensions
)

plot_retention(retention_grouped, retention_history, horizon_days) 

In [ ]:
profiles_with_orders.query('channel != "organic"').groupby('device').agg({'acquisition_cost':'sum', 'revenue':'sum'}).sort_values(by='acquisition_cost', ascending=False)

In [ ]:
profiles_with_orders.query('channel != "organic" & device == "PC"').groupby('channel').agg({'acquisition_cost':'sum', 'revenue':'sum'}).sort_values(by='revenue', ascending=False)

Единственная окупаемая платформа - РС, при том, что у нее самые низкие показатели LTV и конверсии. Видимо, это связано с тем, что на рекламу на РС тратили меньше всего, это и вывело РС в лидеры по окупаемости. 

Платформы iPhone, Мас и Android при высокой конверсии показывают почти такое же удержание пользователей, как РС. У них выше LTV, но и расходы на рекламу очень высокие, и ROI ниже уровня окупаемости на протяжении почти всего лайфтайм пользователя.

### Окупаемость рекламы с разбивкой по странам

In [ ]:
dimensions = ['region']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14
) 

In [ ]:
conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_conversion(conversion_grouped, conversion_history, horizon_days) 

In [ ]:
retention_raw, retention_grouped, retention_history = get_retention(
    profiles, visits, observation_date, horizon_days, dimensions=dimensions
)

plot_retention(retention_grouped, retention_history, horizon_days) 

Графики с разбивкой по странам красноречиво показывают, реклама не окупается только в Америке и что расходы на эту рекламу только растут. В этом регионе высокая конверсия и низкое удержание, высокий LTV. Похоже, реклама в Америке слишком дорогая.

### Окупаемость рекламы с разбивкой по рекламным каналам

In [ ]:
dimensions = ['channel']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14
) 

In [ ]:
conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_conversion(conversion_grouped, conversion_history, horizon_days) 

In [ ]:
retention_raw, retention_grouped, retention_history = get_retention(
    profiles, visits, observation_date, horizon_days, dimensions=dimensions
)

plot_retention(retention_grouped, retention_history, horizon_days) 

In [ ]:
user_revenue.query('channel != "organic"').sort_values(by='acquisition_cost', ascending=False)

In [ ]:
profiles_with_orders.query('channel != "organic" & region != "United States"').groupby('channel').agg({'acquisition_cost':'sum', 'revenue':'sum'}).sort_values(by='acquisition_cost', ascending=False)

In [ ]:
profiles_with_orders.query('channel != "organic"').pivot_table(index=['region','device'], values=['acquisition_cost','revenue'], aggfunc='sum')

TipTop, FaceBoom и AdNonSense - платформы с неокупаемой рекламой. Остальные источники траффика окупаются, и довольно быстро (2-6 дней). Траты на рекламу в TipTop растут несоразмерно остальным платформам. FaceBoom показывает самую высокую конверсию и самое низкое удержание (промокод на первую покупку?). 

## Выводы

Вложения в рекламу не окупаются, а расходы на рекламу увеличивались почти весь анализируемый период. Единственная окупаемая платформа - РС, причина этому - низкие траты на рекламу на этой платформе, что показывает в целом неэффективность решений о размещении рекламы. Платформы iPhone, Мас и Android при высокой конверсии, но и высокой стоимости привлечения показывают отсутствие окупаемости на протяжении почти всего лайфтайм пользователя.

Анализ в разрезе регионов показывает, что реклама не окупается только в Америке, куда выделено 86% рекламного бюджета. 96% этих денег ушло на платформы TipTop и FaceBoom, которые не окупаются. Америка в целом лидер по количеству пользователей, в том числе платящих (78%), однако при высокой конверсии это пользователи с очень низким удержанием. 
Траты на рекламу в TipTop растут несоразмерно остальным платформам, но количество привлеченных пользователей от месяца к месяцу почти не меняется. Потрачено на рекламу в TipTop 54.7 тыс., а выручка от пользователей TipTop составила 69,6 тыс.
Однозначно убыточна реклама в FaceBoom в Америке, на которую было потрачено 32.4 тыс., а выручка составила 25.6 тыс. 

В целом FaceBoom показывает самую высокую конверсию и самое низкое удержание, и привел 40% всех платящих пользователей. Возможно, привлеченным пользователям выдается промокод на первую покупку, а в дальнейшем они ничего не покупают.

Рекомендации для отдела маркетинга для повышения эффективности:

1) расходы на рекламу в FaceBoom следует либо сильно сократить, либо пересмотреть концепцию рекламного предложения, чтобы улучшить показатели удержания пользователя;
2) для американского рынка лучше размещать рекламу на платформах RocketSuperAds, МediaTornado и YRabbit, которые показывали хорошую окупаемость;
3) увеличение расходов на рекламу в TipTop не дает прироста количества пользователей. Видимо, реклама там постоянно дорожает, что объясняет неокупаемость при хорошем удержании и конверсии, так что расходы на эту платформу можно снизить;
4) AdNonSense - платформа с неокупаемой рекламой, которая приводит мало пользователей, от размещения здесь нужно отказатьтся;
5) рекламный бюджет стоит перенаправить на Великобританию, Францию и Германию, которые привлекают более платежеспособного пользователя. Платформы WahooNetBanner, OppleCreativeMedia, LeapBob, lambdaMediaAds показывают хорошик результаты в окупаемости рекламы в европейском направлении. PC - самая перспективная платформа для этих регионов.
